In [ ]:
# Project created with the intent of developing my programming skills
# Pokedoku Solver

# Libraries used: Selenium, bs4, pandas, time
# Selenium is used for web scraping, since pokedoku is a dynamic website, requests didn't work
# bs4 is utilized for html parsing
# pandas is used for fetching data from the csv file
# time is used to ensure all necessary data is scrapped

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

options = Options()
options.add_argument('--headless') # Runs the browser in the background without opening a window

# Ensures the updated version of ChromeDriverManager is installed and creates a Selenium Service instance to run it

service = Service(ChromeDriverManager().install()) 

# Launches Google Chrome using the service

driver = webdriver.Chrome(service=service, options=options)

url = 'https://pokedoku.com/' # The website link that's scrapped

driver.get(url)

time.sleep(5)

html = driver.page_source
driver.quit() # Ends the service once the data has been scrapped

soup = BeautifulSoup(html, 'html.parser') # Parses the HTML file so the necessary data can be easily accessed

# To be able to solve the puzzle, we'll need a list with each square's criteria, that means we'll need each square's data in order,
# from left to right, top to bottom. However, some criteria are saved in the HTML as images, while others are saved as text, making
# it difficult to arrange them in the necessary order.

images = []

image_div = soup.find_all('div', class_='css-1ciybbs') # Saves all image criteria in a list
for div in image_div:
    imgs = div.find_all('img', alt=True)
    for img in imgs:
        images.append(img['alt'])

texts = []

text_div = soup.find_all('div', class_='css-vywf8m') # Saves all text criteria in a list
for div in text_div:
    headers = div.find_all('h2', class_='chakra-heading css-tzo4a1')
    if headers:
        combined_text = ' '.join([header.get_text() for header in headers])
        texts.append(combined_text)

# The criteria we want are now saved, but due to being in two different lists, they are very likely out of order, so we'll look 
# through the HTML file again, with a broader range, and if the element matches one of the elements present in one of the lists, 
# then it'll save it in the criteria list, now in the proper order

criteria = []
seen = set()

for element in soup.find_all(['img', 'h2']): 
    if element.name == 'img' and element.get('alt'):
        if element.get('alt') in images:
            value = element['alt']
            if value not in seen:
                criteria.append(value)
                seen.add(value)
    elif element.name == 'h2' and 'chakra-heading' in element.get('class', []):
        parent_div = element.find_parent('div', class_='css-vywf8m')
        if parent_div:
            h2_texts = [header.get_text() for header in parent_div.find_all('h2', class_='chakra-heading')]
            combined_text = ' '.join(h2_texts).strip()
            if combined_text in texts:
                if combined_text not in seen:
                    criteria.append(combined_text)
                    seen.add(combined_text)

# Now we assign the proper criteria to each of the squares, which will then help browse the csv file

first_square = [criteria[0], criteria[3]]
second_square = [criteria[1], criteria[3]]
third_square = [criteria[2], criteria[3]]
fourth_square = [criteria[0], criteria[4]]
fifth_square = [criteria[1], criteria[4]]
sixth_square = [criteria[2], criteria[4]]
seventh_square = [criteria[0], criteria[5]]
eighth_square = [criteria[1], criteria[5]]
ninth_square = [criteria[2], criteria[5]]

df = pd.read_csv('pokemon.csv')

criteria_list = [
    first_square, 
    second_square,
    third_square,
    fourth_square,
    fifth_square,
    sixth_square,
    seventh_square,
    eighth_square,
    ninth_square
]

# Function created to check the lines of the dataframe, and see if the line matches the criteria in each square, if so, it attaches 
# it to a dictionary

def row_matches(row, search_criteria):
    row_values = row.astype(str).str.lower().tolist()
    return all(term.lower() in row_values for term in search_criteria)


# Finally, it goes through each square and their criteria, finds the first row in the dataframe that matches it, and attaches it
# to a dictionary, giving us the answers to the puzzle

results = {}

for i, search_criteria in enumerate(criteria_list, start=1):
    matching_row = df[df.apply(lambda row: row_matches(row, search_criteria), axis=1)].head(1)
    if not matching_row.empty:
        results[f'Square {i} ({', '.join(search_criteria)})'] = matching_row.iloc[0]['Name']
    else:
        results[f'Square {i} ({', '.join(search_criteria)})'] = 'No Match Found'

for key, value in results.items():
    print(f'{key}: {value}')

Square 1 (psychic, fire): Victini
Square 2 (MIDDLE EVOLUTION, fire): Charmeleon
Square 3 (ground, fire): Numel
Square 4 (psychic, normal): Girafarig
Square 5 (MIDDLE EVOLUTION, normal): Pidgeotto
Square 6 (ground, normal): Diggersby
Square 7 (psychic, PALDEA + AREA ZERO DLC): Armarouge
Square 8 (MIDDLE EVOLUTION, PALDEA + AREA ZERO DLC): Floragato
Square 9 (ground, PALDEA + AREA ZERO DLC): Wooper
